In [ ]:
import tensorboard

In [ ]:
import os

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import tensorflow
from keras.callbacks import TensorBoard
from tensorflow.keras.metrics import *
from tensorflow.keras.preprocessing.image import ImageDataGenerator

import src.data.Dataset as dt

In [ ]:
required_paths = ["/ai4eutils", "/CameraTraps", "/yolov5"]
python_path = os.environ.get("PYTHONPATH", "")
root_path = os.getcwd()

for path in required_paths:
    if not any(p.endswith(path) for p in python_path.split(":")):
        python_path += f":{root_path}/data/external{path}"

os.environ["PYTHONPATH"] = python_path

!echo "PYTHONPATH: $PYTHONPATH"

In [ ]:
IMAGES_DATASET_PATH = os.path.abspath("./data/raw/emptyNonEmptyDataset_ETL")

TRAIN_SAMPLES_CSV = os.path.abspath(
    "./data/raw/emptyNonEmptyDataset_ETL/28570Train.csv"
)
VALIDATION_SAMPLES_CSV = os.path.abspath(
    "./data/raw/emptyNonEmptyDataset_ETL/28570Validation.csv"
)
TEST_SAMPLES_CSV = os.path.abspath("./data/raw/emptyNonEmptyDataset_ETL/28570Test.csv")

print(f"IMAGES_DATASET_PATH:    {IMAGES_DATASET_PATH}")
print(f"TRAIN_SAMPLES_CSV:      {TRAIN_SAMPLES_CSV}")
print(f"VALIDATION_SAMPLES_CSV: {VALIDATION_SAMPLES_CSV}")
print(f"TEST_SAMPLES_CSV:       {TEST_SAMPLES_CSV}")

In [ ]:
train_dataframe = dt.load_from_csv(TRAIN_SAMPLES_CSV)
validation_dataframe = dt.load_from_csv(VALIDATION_SAMPLES_CSV)
test_dataframe = dt.load_from_csv(TEST_SAMPLES_CSV)

In [ ]:
train_dataframe["file_name"] = train_dataframe["file_name"].apply(
    lambda x: os.path.join(IMAGES_DATASET_PATH, x)
)
train_dataframe["binary_label"] = train_dataframe["binary_label"].astype(str)

validation_dataframe["file_name"] = validation_dataframe["file_name"].apply(
    lambda x: os.path.join(IMAGES_DATASET_PATH, x)
)
validation_dataframe["binary_label"] = validation_dataframe["binary_label"].astype(str)

test_dataframe["file_name"] = test_dataframe["file_name"].apply(
    lambda x: os.path.join(IMAGES_DATASET_PATH, x)
)
test_dataframe["binary_label"] = test_dataframe["binary_label"].astype(str)

In [ ]:
IMG_SIZE = (224, 224)
BATCH_SIZE = 32
EPOCHS = 25
VERSION = 2.0

train_datagen = ImageDataGenerator(
    preprocessing_function=tensorflow.keras.applications.vgg16.preprocess_input
)
normal_datagen = ImageDataGenerator(
    preprocessing_function=tensorflow.keras.applications.vgg16.preprocess_input
)

train_images = train_datagen.flow_from_dataframe(
    dataframe=train_dataframe,
    x_col="file_name",
    y_col="binary_label",
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode="binary",
    seed=42,
)

validation_images = normal_datagen.flow_from_dataframe(
    dataframe=validation_dataframe,
    x_col="file_name",
    y_col="binary_label",
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode="binary",
    seed=42,
)

test_images = normal_datagen.flow_from_dataframe(
    dataframe=test_dataframe,
    x_col="file_name",
    y_col="binary_label",
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode="binary",
    shuffle=False,
)

In [ ]:
import numpy as np
import matplotlib.pyplot as plt


def show_random_images_from_generator(generator, title):
    # Obtener imágenes y etiquetas del generador
    images, labels = next(generator)

    # Revertir el preprocesamiento de VGG16
    mean = np.array([103.939, 116.779, 123.68])  # Medias de ImageNet
    images = images + mean  # Revertir la normalización por medias
    images = images[..., ::-1]  # Convertir de BGR a RGB
    images = np.clip(images, 0, 255)  # Asegurarse de que los valores estén en [0, 255]

    # Visualizar las imágenes
    plt.figure(figsize=(15, 5))
    for i in range(min(10, len(images))):  # Mostrar un máximo de 10 imágenes
        plt.subplot(2, 5, i + 1)
        plt.imshow(images[i].astype("uint8"))  # Convertir a uint8 para visualización
        plt.title(f"Label: {int(labels[i])}")
        plt.axis("off")
    plt.suptitle(title)
    plt.show()


# Mostrar ejemplos de cada conjunto
show_random_images_from_generator(train_images, "Training Set")
show_random_images_from_generator(validation_images, "Validation Set")
show_random_images_from_generator(test_images, "Test Set")

In [ ]:
base_model = tensorflow.keras.applications.VGG16(
    weights="imagenet", include_top=False, input_shape=(IMG_SIZE[0], IMG_SIZE[1], 3)
)
base_model.trainable = False

# Construir el modelo básico
model = tensorflow.keras.models.Sequential(
    [
        base_model,
        tensorflow.keras.layers.Flatten(),
        tensorflow.keras.layers.Dense(4096, activation="relu"),
        # tensorflow.keras.layers.Dropout(0.5),
        tensorflow.keras.layers.Dense(4096, activation="relu"),
        # tensorflow.keras.layers.Dropout(0.5),
        tensorflow.keras.layers.Dense(1, activation="sigmoid"),
    ],
    name=f"VGG16_v.{VERSION}",
)

model.compile(
    optimizer="adam",
    loss="binary_crossentropy",
    metrics=[
        "accuracy",
        "binary_accuracy",
        Precision(name="presision"),
        Recall(name="recall"),
        AUC(name="auc"),
        TruePositives(name="tp"),
        FalsePositives(name="fp"),
        TrueNegatives(name="tn"),
        FalseNegatives(name="fn"),
    ],
)

model.summary()

In [ ]:
history = model.fit(
    train_images,
    validation_data=validation_images,
    epochs=EPOCHS,
    callbacks=[TensorBoard(log_dir=f"./logs/VGG16/v.{VERSION}")],
)

In [ ]:
model.save(f"./models/VGG16/VGG16_v.{VERSION}.h5")

In [ ]:
history_df = pd.DataFrame(history.history)
history_df.to_csv(f"./logs/VGG16/v.{VERSION}/train_validation_history.csv", index=False)

In [ ]:
results = history.model.evaluate(test_images)

metric_names = history.model.metrics_names
evaluation_results = {
    ("test_" + name): value for name, value in zip(metric_names, results)
}
evaluation_df = pd.DataFrame([evaluation_results])
evaluation_df.to_csv(f"./logs/VGG16/v.{VERSION}/test_history.csv", index=False)
